# Laboratorio 2
## Microestructuras y Sistemas de Trading
### Valentina Escobar y Pablo Mendoza

El presente laboratorio tiene como objetivo analizar el comportamiento de un ETF de acuerdo a la conformación de este y al movimiento de los precios que se presentaron entre el 31 de marzo del 2017 hasta el 29 de marzo del 2018 con el fin de identificar que tan conveniente es para un inversionista rebalancear diariamente durante un año comprando $n$ acciones cuando el rendimiento diario sea de -1%. Se contrastará el capital inicial con el capital final deshaciendo todas las pocisiones del portafolo el 1 de Marzo de 2018. Se realizan diferentes medidas de desempeño que permitan comparar los portafolios resultantes.

## Librerías y Funciones 

In [1]:
# Librerias
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
quandl.ApiConfig.api_key = "My9ygJsz7yTDx_9jcneH" # Quandl API key
fechas = list(['2017-03-31','2018-03-29'])

In [45]:
data_csv = pd.read_csv('/Users/pablomendoza/Documents/ITESO/Trading/Lab_1/Trading/Data/IJH_holdings-1.csv', skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'])
tick = list(data_csv.index)
weights = data_csv['Weight (%)']
#del data_csv
def Download_QuandL(fechas:'Dates_list ', tick:'Tickers of Stock'):
    dates = { 'gte': fechas[0], 'lte': fechas[1] }
    column = { 'columns': ['date', 'Open', 'Close']}
    data = quandl.get_table('WIKI/PRICES', qopts = column, ticker = tick, date = dates, paginate=True)# ticker = 'WIKI/AAPL' #'AAPL.US'
    data = data.iloc[::-1]
    data = data.reset_index()
    data = data.drop(['None'],1)
    return data
def All_Data(fechas:'Dates_list ', tick:'Ticker'):
    d = {} # Crea un Diccionario Vacío
    for i in range(len(tick)): # ciclo que descarga los datos de Quandl y los guarda como DataFrames en un Diccionario
        nom = str(tick[i])
        d["_{0}".format(nom)] = Download_QuandL(fechas, tick[i])
    return d
def Rebalanceo(df, Com, Cap_I, R_R, n_a, we, op, t):
    """
    Esta funcion hace los rebalanceos de cada activo de forma diaria por una año
    :param df: DataFrame del Activo
    :param Com: Comisión
    :param Cap_I: Capital Inicial
    :param R_R: Rendimiento de Rebalanceo
    :param n_a: Acciones que se compran al rebalancear
    :param we: Weight of each stock on the porfolio
    :param op: Dataframe to save operation
    :param t: ticker
    :return: 
    """
    a = (df['close']/df['open'])-1
    P_se = list(df['open']*.99)
    add_s = list()
    Opp = list()
    comi = list()
    In_shares = np.floor((we*Cap_I)/df['close'][0])
    for i in range(len(df)):
        if a.iloc[i] < R_R:
            add_s.append(n_a)
            Opp.append('Compra')
            comi.append(Com*df['close'].iloc[i]*n_a)
        else:
            add_s.append(0)
            Opp.append('-')
            comi.append(0)
    rt = list(np.ones(len(df)))
    rt = pd.DataFrame([t if x % 2 else Pass for x in rt], columns = ['Ticker'])      
    df = df.join(pd.DataFrame(add_s, columns = ['# Added Shares']))
    df = df.join(pd.DataFrame(Opp, columns = ['Operacion']))
    df = df.join(pd.DataFrame(comi, columns = ['Comisión']))
    df = df.join(pd.DataFrame(P_se, columns = ['Precio de Señal']))
    Tot_Acciones = pd.DataFrame(list(df['# Added Shares'].cumsum()), columns = ['Total Shares'])+In_shares
    df = df.join(Tot_Acciones)
    Val_to_port = pd.DataFrame(list(df['close']*df['Total Shares']), columns = ['Value to ETF'])
    df = df.join(Val_to_port)
    df = df.join(rt)
    frames = [op, df[df.Operacion == 'Compra']]
    op = pd.concat(frames)
    Cap_I = Cap_I - 
    
    
    
    return df, op
def Clean_Dicc(dic:'Diccionario a limpiar',sizes:'Tamaño deseado del DF'):
    y = list(dic.keys())
    for h in y:
        if len(dic[h])!=248:
            del dic[h]
    return dic

In [4]:
Data = All_Data(fechas, tick)

In [5]:
d_full = Clean_Dicc(Data,248)

In [122]:
tick = list(d_full.keys())
d2 = {}
op = pd.DataFrame(columns=['date', 'open', 'close', '# Added Shares', 'Operacion', 'Comisión', 'Precio de Señal', 'Total Shares', 'Value to ETF', 'Ticker'''])
for j in range(len(tick)):
    tabla , op = Rebalanceo(d_full[tick[j]],0.0003,1000000,-0.01,10,weights[j], op, tick[j])
    d2["Data{0}".format(j)] = tabla
op_2 = op.groupby('date')
#for key, item in op_2:
    #print(op_2.get_group(key), "\n\n")
Key_op_2 = list()
for key, item in op_2:
    Key_op_2.append(key)
# Data: tiene la información de todas las transacciones para cada activo en cada día
# op: tiene la información de todas las transacciones de compra para cada activo
# op_2: tiene la información de todas las transacciones de compra agrupada por día
l_comi = list()
diff_capital = list()
for w in range(len(op_2)):
    l_comi.append(float(pd.DataFrame(op_2.get_group(Key_op_2[w])['Comisión']).sum()))
    diff_capital.append(float(pd.DataFrame(pd.DataFrame(op_2.get_group(Key_op_2[w])['Precio de Señal'])['Precio de Señal']*pd.DataFrame(op_2.get_group(Key_op_2[w])['# Added Shares'])['# Added Shares']).sum()))

In [123]:
diff_capital

[15963.849000000007,
 70458.69599999998,
 29681.685,
 143971.443,
 6066.621,
 14661.504,
 10806.146999999999,
 2428.6680000000006,
 107409.75299999994,
 85591.34099999997,
 5616.665999999999,
 5818.032,
 19671.398999999998,
 26563.779000000013,
 25630.605,
 32254.992000000002,
 23168.376,
 28655.649,
 33705.639,
 106675.37099999997,
 31266.47700000001,
 67300.59599999999,
 44327.052,
 36699.59700000001,
 25448.54399999999,
 38798.19900000001,
 62471.524499999985,
 6506.0819999999985,
 53821.547999999995,
 35909.37899999999,
 10391.139000000001,
 30856.41899999999,
 106161.85800000001,
 15940.484999999999,
 6537.465,
 18130.959000000003,
 54695.81699999999,
 17138.385,
 24747.030000000002,
 18287.676000000003,
 26782.668,
 23672.681999999997,
 4480.839,
 15369.947999999997,
 57000.537,
 27523.386,
 24759.702,
 14042.853000000003,
 44762.75100000001,
 37487.834999999985,
 19380.636000000002,
 42267.158999999985,
 13704.767999999998,
 13355.595000000001,
 26639.514,
 87204.34800000001,
 7